In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from pydub import AudioSegment

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import MaxAbsScaler

##### Função recebe a quantidade de arquivos por classe, o nome da pasta e o nome das classes. Cria o caminho e salva na pasta lista_arquivos. Depois percorre os arquivos, abrindo cada aúdio, extraindo cada informação, aplica um filtro para remover os outliers, deixando apenas as informações menores que -20 e maiores que 20. Sabendo da necessidade de deixar todos os audios com o mesmo tamanho de array, uso a formula int(shape[0]) / 1000, e pego 1000 dados de cada audio, de forma a percorre-los 

In [2]:
def transform_audios(quantidade_arquivos, pasta, info):

    lista_arquivos = []

    lista_audios = []

    lista_classe = []

    for palavra in info:

        for i in range(quantidade_arquivos):
    
            string = f"{pasta}/{0}{i+1}_{palavra}.mp3"
            lista_arquivos.append(string)

            if palavra == "abrir":
                lista_classe.append(0)

            elif palavra == "compilar":
                lista_classe.append(1)
            
            elif palavra == "executar":
                lista_classe.append(2)

            else:
                lista_classe.append(3)

    for arquivo in lista_arquivos:
    
        sound = AudioSegment.from_mp3(arquivo)
        raw_data = sound._data

        audio = []
        
        for si in range(len(raw_data)//2):
            sample = int.from_bytes(raw_data[si * 2 : si * 2 + 2], 'little', signed=True)
            if sample < -30 or sample > 30:
                audio.append(sample)
        
        audio = np.array(audio)
        
        shape = audio.shape
  
        calculo = int(shape[0]) / 4150

        calculo = int(calculo)
        
        audio = audio[::calculo]
        
        audio = audio[:4150]
        
        lista_audios.append(audio)

    df = pd.DataFrame(lista_audios)
    df["classe"] = lista_classe

    return df

##### Aplicando a função nos arquivos de treinamento e criando x e y para treinamento. Aplicada a função MaxAbsScaler, para normalizar os dados.

In [3]:
quantidade_arquivos_por_classe = 8

info = ["abrir", "compilar", "executar", "testar"]

pasta = "audios_treinamento"

arquivos_audio_treinamento = transform_audios(quantidade_arquivos_por_classe, pasta, info)

In [5]:
x = arquivos_audio_treinamento.drop(['classe'], axis= 1)

In [8]:
# x

In [6]:
y = arquivos_audio_treinamento["classe"]

In [170]:
# y

In [9]:
scaler = MaxAbsScaler().fit(x)  

x = scaler.transform(x)

In [11]:
# x

##### Aplicando a função nos arquivos de teste e criando x_test e y_test, também aplicada função para normalização.

In [12]:
quantidade_arquivos_por_classe = 2

info = ["abrir", "compilar", "executar", "testar"]

pasta = "audios_teste"

arquivos_audio_teste = transform_audios(quantidade_arquivos_por_classe ,pasta, info)

In [13]:
x_test = arquivos_audio_teste.drop(['classe'], axis= 1)

In [175]:
# x_test

In [14]:
y_test = arquivos_audio_teste["classe"] 

In [177]:
# y_test

In [15]:
scaler = MaxAbsScaler().fit(x_test)

x_test = scaler.transform(x_test)

In [179]:
# x_test

##### Treinando o algoritmo e testando com os arquivos de teste.

In [16]:
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(x, y)

y_pred = knn_model.predict(x_test)

In [17]:
print(f"Testando com dados de teste. Score: {knn_model.score(x_test, y_test) * 100} %")

Testando com dados de teste. Score: 50.0 %


In [18]:
y_pred = pd.DataFrame(y_pred)

y_pred = y_pred.astype(int)

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.40      0.57         5
           2       1.00      0.67      0.80         3
           3       0.00      0.00      0.00         0

    accuracy                           0.50         8
   macro avg       0.50      0.27      0.34         8
weighted avg       1.00      0.50      0.66         8



/home/dev/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/dev/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/dev/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Treinando o algoritmo, Embaralhando os dados e testando com dados de treinamento.

In [19]:
knn_model = KNeighborsClassifier(n_neighbors=1)
knn_model.fit(x, y)

arquivos_audio_teste = arquivos_audio_treinamento.sort_values(0) 

In [20]:
x_test = arquivos_audio_teste.drop(['classe'], axis= 1)
y_test = arquivos_audio_teste["classe"] 

In [185]:
# x_test

In [186]:
# y_test

In [21]:
y_pred = knn_model.predict(x_test)

print(f"Testando com dados de treinamento. Score: {knn_model.score(x_test, y_test) * 100} %")

Testando com dados de treinamento. Score: 100.0 %


In [22]:
y_pred = pd.DataFrame(y_pred)

y_pred = y_pred.astype(int)

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00         8
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00         8

    accuracy                           1.00        32
   macro avg       1.00      1.00      1.00        32
weighted avg       1.00      1.00      1.00        32

